## Extra homework workshop DLT DE Zoomcamp
### Using DLT to move data from a bunch of parquet files to BIGQUERY
For this homework we will be using the Yellow Taxi Trip Records for January 2024 - June 2024 NOT the entire year of data Parquet Files from the New York City Taxi Data found here:
https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page


* First I have to install the package

In [29]:
%%capture
!pip install dlt[bigquery]

Declare the url_parquet with the webpage and extract the urls from that web

In [ ]:
import dlt
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import BytesIO

url_parquet = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

def get_parquet_urls():
    """Get the urls from the TLC NYC webpage data"""
    response = requests.get(url_parquet)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all("a", href=True)

    parquet_urls = [
        link["href"] for link in links if link["href"].endswith(".parquet")
    ]

    return parquet_urls




In [ ]:
#get the urls and insert into a variable urls_list
urls_list = get_parquet_urls()

# to see the url
urls_list[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet'

Insert into the parquet_urls list the yellow_tripdata from January 2024 to June 2024

In [ ]:
parquet_urls = [f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-0{str(month)}.parquet' for month in range(1,7)]
parquet_urls

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet']

Connect to to download the data

In [ ]:
@dlt.resource(name="ny_taxi_data", write_disposition="replace")
def ny_taxi():

  for url in parquet_urls:
    response = requests.get(url)
    response.raise_for_status()

    df = pd.read_parquet(BytesIO(response.content))
    yield df.to_dict(orient="records")

declare the credentials from bigquery

In [ ]:
import os
from google.colab import userdata

os.environ["DESTINATION__BIGQUERY__CREDENTIALS"] = userdata.get('BIGQUERY_CRED')

Create the pipeline in order to insert the data into bigquery

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="taxi_data",
    destination="bigquery",
    dataset_name="taxi_ny_data_yellow",
    dev_mode=True,
)

info = pipeline.run(ny_taxi)
print(info)

Pipeline taxi_data load step completed in 1 minute and 29.25 seconds
1 load package(s) were loaded to destination bigquery and into dataset taxi_ny_data_yellow_20250217075307
The bigquery destination used terraform-runner@celtic-surface-447817-d0.iam.gserviceaccount.com@celtic-surface-447817-d0 location to store data
Load package 1739821987.71076 is LOADED and contains no failed jobs
